# Regression using satellite images

In [2]:
import sklearn.preprocessing as preprocessing
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from sklearn.metrics import r2_score

## train_model

In [2]:
def train_model(dataloader, model, optimizer, criterion,  out_scaler, feature_scaler, epochs=20, print_step=1000):
    for epoch in range(epochs): 

        running_loss = 0.0
        for i, sample_batched in enumerate(dataloader):
            
            expected_outputs = out_scaler.transform(sample_batched['out'])
            features = feature_scaler.transform(sample_batched['features'])
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model(sample_batched['image'].to(device), features.to(device))
            loss = criterion(outputs, expected_outputs)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_step == print_step-1:    # print every 500 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_step))
                running_loss = 0.0
    
    print('Finished Training')

## test_model

In [13]:
def test_model(dataloader, model):
    with torch.no_grad():
        expected_outputs = []
        outputs = []
        for sample_batched in iter(dataloader):
            expected_outputs += out_scaler.transform(sample_batched['out'])
            features = feature_scaler.transform(sample_batched['features'])

            outputs += net(sample_batched['image'].to(device), features.to(device))
    print(expected_outputs)
    print(outputs)
    print(f"R2 score: {r2_score(outputs, expected_outputs)}")

## BiodiversityDataset

In [4]:
class BiodiversityDataset(Dataset):

    def __init__(self, csv_file, img_dir, image_transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with features.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a image.
        """
        self.features = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.image_transform = image_transform
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        lon = self.features.loc[idx].longitude
        lat = self.features.loc[idx].latitude
        img_name = str(lon) + "_" + str(lat) + ".jpg"
        img_path = os.path.join(self.img_dir, img_name)
        
        image = Image.open(img_path)
        if self.image_transform:
            image = self.image_transform(image)

        out = torch.from_numpy(np.array([self.features.loc[idx]['habitat_richness']])).detach().clone().reshape([-1])
        features = self.features.drop(columns=['habitat_richness', 'longitude', 'latitude']).loc[idx].values
        features = torch.from_numpy(features.astype('float').reshape(-1, 46)).detach().clone().reshape([-1])
        sample = {'image': image, 'features': features, 'out': out}

        return sample
    
    def get_feature(self):
         return torch.from_numpy(self.features.drop(columns=['habitat_richness', 'longitude', 'latitude']).values).detach().clone()
    
    def get_out(self):
        return torch.from_numpy(self.features['habitat_richness'].values).detach().clone()
    

## FeatureImageNet

In [5]:
class FeatureImageNet(nn.Module):

    def __init__(self, image_net, feature_input_size, output_size):
        super(FeatureImageNet, self).__init__()
        self.image_net = image_net
    
        self.feat_fc1 = nn.Linear(feature_input_size, 46)  
        self.feat_fc2 = nn.Linear(46, 100)
        self.feat_fc3 = nn.Linear(100, 128)

        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, images, features):
        x_features = F.relu(self.feat_fc1(features))
        x_features = F.relu(self.feat_fc2(x_features))
        x_features = F.relu(self.feat_fc3(x_features))
        
        x_image = self.image_net(images)
      
        x = torch.cat([x_features, x_image] , dim=1, out=None)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x


## PyTMinMaxScaler

In [6]:
class PyTMinMaxScaler():
    """
    Transforms each channel to the range [0, 1].
    """
    def transform(self, tensor):
        tensor.mul_(self.scale).sub_(self.subtract)
        return tensor.float() 
    
    def fit(self, tensor):
        t = tensor.detach().clone()
        self.scale = 1.0 / (t.max(dim=0, keepdim=True)[0] - t.min(dim=0, keepdim=True)[0])
        self.scale[self.scale == float("Inf")] = 0
        t.mul_(self.scale)
        self.subtract = t.min(dim=0, keepdim=True)[0]
        
    def fit_transform(self, tensor):
        self.scale = 1.0 / (tensor.max(dim=0, keepdim=True)[0] - tensor.min(dim=0, keepdim=True)[0])
        self.scale[self.scale == float("Inf")] = 0
        tensor.mul_(self.scale)
        self.subtract = tensor.min(dim=0, keepdim=True)[0]
        tensor.sub_(self.subtract)
        return tensor.float() 
        

## Loading data

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [8]:
data_dir = "../Dataset/images"

image_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

feature_scaler = PyTMinMaxScaler()
out_scaler = PyTMinMaxScaler()

dataset = BiodiversityDataset(data_dir+"/test.csv", data_dir+"/test", image_transform)

feature_scaler.fit(dataset.get_feature())
out_scaler.fit(dataset.get_out().reshape([-1, 1]))


resnet = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
resnet.fc = nn.Linear(512, 128)

net = FeatureImageNet(image_net=resnet, feature_input_size=46, output_size=1)
net.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


Using cache found in C:\Users\minoc/.cache\torch\hub\pytorch_vision_v0.6.0


In [9]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

In [15]:
train_model(dataloader=dataloader, model=net, optimizer=optimizer,  out_scaler=out_scaler, 
            feature_scaler=feature_scaler, criterion=criterion, epochs=20, print_step=2)

[1,     2] loss: 0.039
[2,     2] loss: 0.016
[3,     2] loss: 0.254
[4,     2] loss: 0.035
[5,     2] loss: 0.236
[6,     2] loss: 0.157
[7,     2] loss: 0.103
[8,     2] loss: 0.197
[9,     2] loss: 0.279
[10,     2] loss: 0.196
[11,     2] loss: 0.195
[12,     2] loss: 0.147
[13,     2] loss: 0.076
[14,     2] loss: 0.209
[15,     2] loss: 0.059
[16,     2] loss: 0.136
[17,     2] loss: 0.049
[18,     2] loss: 0.061
[19,     2] loss: 0.183
[20,     2] loss: 0.033
Finished Training


In [16]:
test_model(dataloader, net)

[tensor([0.9366]), tensor([0.9366]), tensor([1.]), tensor([0.]), tensor([1.]), tensor([0.]), tensor([0.4845])]
[tensor([0.8534]), tensor([0.3932]), tensor([1.0400]), tensor([0.0380]), tensor([0.7545]), tensor([0.1191]), tensor([0.8299])]
R2 score: 0.45784441850545543
